In [1]:
from xgboost import XGBRegressor
import sklearn
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
import sklearn.ensemble, sklearn.ensemble, sklearn.svm

In [2]:
data = pd.read_csv("ready.csv")

data = data.replace(to_replace="N/", value=np.NAN)

X = data.drop(columns="rating")
y = data["rating"]
X = X[~y.isna()]
y = y[~y.isna()].to_numpy()

X_copy = X.copy()
y_copy = y.copy()

X = X.iloc[:,3:]
knnI = KNNImputer(missing_values=np.NAN)
X = knnI.fit_transform(X)

s = sklearn.preprocessing.StandardScaler()
X = s.fit_transform(X)

X_tr, X_te, y_tr, y_te = sklearn.model_selection.train_test_split(X, y, train_size=0.75)

In [3]:
xgb = XGBRegressor()
xgb.fit(X_tr, y_tr)
xgb.score(X_te, y_te)

0.5713115330247185

In [4]:
rF = sklearn.ensemble.RandomForestRegressor()
rF.fit(X_tr, y_tr)
rF.score(X_te, y_te)

0.5954586210843498

In [5]:
lR = sklearn.linear_model.LinearRegression()
lR.fit(X_tr, y_tr)
lR.score(X_te, y_te)

0.6047401804752432

In [6]:
svr = sklearn.svm.SVR()
svr.fit(X_tr, y_tr)
svr.score(X_te, y_te)

0.6099552861322528

In [7]:
xgb = XGBRegressor()
xgb.fit(X_tr, y_tr)
scores = xgb.predict(X)

In [8]:
final = pd.concat((X_copy, pd.DataFrame(y_copy, columns=["rating"]), pd.DataFrame(scores, columns=["score"])), axis=1)
final = final.sort_values("score", ascending=False).drop_duplicates("name")

final["total_tags"] = [sum(final[final.columns[6:-2]].iloc[i]) for i in range(final.shape[0])]
final[final.total_tags > 5]

,name,department,school,take_again,difficulty,avg_gpa,Clear grading criteria,Respected,Get ready to read,Skip class? You won't pass.,...,Inspirational,Amazing lectures,Caring,Gives good feedback,Participation matters,Hilarious,Lots of homework,rating,score,total_tags
1920,"YOUSSEFPOUR, H.",Mathematics,UC Irvine,92,3.9,2.609063,5,6,0,0,...,0,0,3,0,0,3,0,5.3,5.249608,29
853,"BOBIER, C.",Philosophy,UC Irvine,100,3.0,3.232500,3,0,0,0,...,0,0,0,6,4,0,0,5.0,5.182883,13
206,"SPERLING, G.",Psychology,UC Irvine,100,4.1,3.395000,0,0,3,0,...,4,0,3,0,0,0,0,2.7,5.147331,10
1214,"SENATRO, L.",English,UC Irvine,100,3.0,3.538000,3,0,0,0,...,0,0,3,3,0,0,0,5.0,5.135935,9
1308,"LEWIN, K.",English,UC Irvine,100,3.3,3.042778,6,4,0,0,...,3,4,6,14,3,5,4,5.0,5.119767,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,"HIRSHLEIFER, S.",Accounting,UC Irvine,0,4.8,3.307143,0,0,7,0,...,0,0,0,0,3,0,0,1.3,1.261190,10
1244,"PREECE, D.",Biomedical,UC Irvine,0,4.5,3.083333,0,0,0,0,...,0,0,0,0,0,0,0,1.0,1.222557,6
1129,"QUIROZ, B.",Education,UC Irvine,0,4.6,3.185000,0,0,6,0,...,0,0,0,0,0,0,0,1.2,1.184188,19
814,"JACOBY, A.",History,UC Irvine,0,5.0,2.946250,0,0,3,5,...,0,0,0,0,0,0,0,1.0,1.074909,14


In [9]:
final.to_csv("final_regression.csv", index=False)

In [149]:
grades = pd.read_csv("course_grades.csv")
scores = pd.read_csv("final_regression.csv")

s = sklearn.preprocessing.MinMaxScaler()
grades["avg_gpa"] = s.fit_transform(np.array(grades["avg_gpa"]).reshape(-1,1))
scores["score"] = s.fit_transform(np.array(scores["score"]).reshape(-1,1))

def class_score(professor, course):
    return scores[scores.name == professor]["score"].iloc[0] + grades[grades.course_name == course]["avg_gpa"].iloc[0] / 2

def get_schedule_score(schedule):
    scores = []
    for i in schedule.courses:
        scores.append(class_score(i.professors[0], i.course_name))

    return sum(scores) /  len(scores)